In [1]:
#首先进行数据集的读入，然后进行训练集和交叉测试集的划分

import pandas as pd
import numpy as np

train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")

train_set.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
#对训练集划分输入和输出
train_all_x = np.array(train_set.iloc[:, 1:].values, dtype = np.float32)
train_all_y = np.array(train_set.iloc[:, 0].values, dtype = np.float32)

#对输出采用one-hot编码
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
train_all_y_onehot = enc.fit_transform(train_all_y.reshape(-1, 1)).toarray()
train_all_y_onehot[:10, :]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [3]:
#正式划分训练集和交叉测试集
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_all_x, train_all_y_onehot, test_size = 0.1, random_state = 0)
test_x[:10, :]
test_x.shape

(4200, 784)

In [4]:
#确认train_x, train_y, test_x和test_y的维度
print("train_x shape is : " + str(train_x.shape))
print("train_y shape is : " + str(train_y.shape))
print("test_x shape is : " + str(test_x.shape))
print("test_y shape is : " + str(test_y.shape))

train_x shape is : (37800, 784)
train_y shape is : (37800, 10)
test_x shape is : (4200, 784)
test_y shape is : (4200, 10)


In [5]:
#导入必要的依赖
import tensorflow as tf
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from tensorflow.python.framework import ops

In [6]:
#创建placeHolders
# GRADED FUNCTION: create_placeholders

def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    ### START CODE HERE ### (≈2 lines)
    X = tf.placeholder(tf.float32, (None, n_H0, n_W0, n_C0 ))
    Y = tf.placeholder(tf.float32, (None, n_y ))
    ### END CODE HERE ###
    
    return X, Y

In [7]:
#定义相关所需函数，用于权值初始化以及网络基本机构的定义
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.0001)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding = 'SAME')

#池化层，滤波器的大小为2 x 2
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [8]:
#正式设计网络架构

#设计第一层卷积层，滤波器大小为5 x 5，共32个滤波器
W1 = weight_variable([5, 5, 1, 32])
b1 = bias_variable([32])

#设计第二层卷积层，滤波器大小为5 x 5， 共64个滤波器
W2 = weight_variable([5, 5, 32, 64])
b2 = bias_variable([64])

#设计第一个全连接层，经过两次pooling层后从28 x 28变成了7 x 7的大小
W_flat1 = weight_variable([7*7*64 ,1024])
b_flat1 = bias_variable([1024])

#设计第一个全连接层
W_flat2 = weight_variable([1024, 10])
b_flat2 = bias_variable([10])

In [9]:
#开始进行网络层间连接

x = tf.placeholder("float",shape=[None,784])
y_ = tf.placeholder("float",shape=[None,10])

train_x_input = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(train_x_input, W1) + b1)
h_pool1 = max_pool_2x2(h_conv1)

h_conv2 = tf.nn.relu(conv2d(h_pool1, W2) + b2)
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_flat1) + b_flat1)

#进行随机失活，避免过拟合
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_out = tf.nn.softmax(tf.matmul(h_fc1_drop, W_flat2) + b_flat2)

In [10]:
#设置优化的方法
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_out))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#tf.argmax是对矩阵中的行或列计算最大值，并返回对应的下标
correct_prediction = tf.equal(tf.argmax(y_out, 1),tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
#开始设置每次训练的样本

epochs_completed = 0                         #用于记录完成的迭代次数
index_in_epoch = 0
num_examples = train_x.shape[0]

# serve data by batches
def next_batch(batch_size):
    
    global train_x
    global train_y
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)                           #打乱样本顺序
        train_x = train_x[perm]
        train_y = train_y[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_x[start:end], train_y[start:end]

In [12]:
#设置一些全局的变量

# settings
LEARNING_RATE = 1e-5
# set to 20000 on local environment to get 0.99 accuracy
TRAINING_ITERATIONS = 5000        
    
DROPOUT = 0.5
BATCH_SIZE = 50

# set to 0 to train on all available data
VALIDATION_SIZE = 2000

# image number to output
IMAGE_TO_DISPLAY = 10

In [13]:
#开始训练模型

# visualisation variables
train_accuracies = []
validation_accuracies = []
x_range = []

display_step=1

for i in range(TRAINING_ITERATIONS):

    #get new batch
    batch_xs, batch_ys = next_batch(BATCH_SIZE)

    # check progress on every 1st,2nd,...,10th,20th,...,100th... step
    if i%display_step == 0 or (i+1) == TRAINING_ITERATIONS:
        
        train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys, keep_prob:1.0})       
        if(VALIDATION_SIZE):
            validation_accuracy = accuracy.eval(feed_dict={ x: test_x[0:BATCH_SIZE], 
                                                            y_: test_y[0:BATCH_SIZE], 
                                                            keep_prob: 1.0})                                  
            print('training_accuracy / validation_accuracy => %.2f / %.2f for step %d'%(train_accuracy, validation_accuracy, i))
            
            validation_accuracies.append(validation_accuracy)
            
        else:
             print('training_accuracy => %.4f for step %d'%(train_accuracy, i))
        train_accuracies.append(train_accuracy)
        x_range.append(i)
        
        # increase display_step
        if i%(display_step*10) == 0 and i:
            display_step *= 10
    # train on batch
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: DROPOUT})

training_accuracy / validation_accuracy => 0.08 / 0.06 for step 0
training_accuracy / validation_accuracy => 0.14 / 0.10 for step 1
training_accuracy / validation_accuracy => 0.04 / 0.10 for step 2
training_accuracy / validation_accuracy => 0.12 / 0.10 for step 3
training_accuracy / validation_accuracy => 0.14 / 0.12 for step 4
training_accuracy / validation_accuracy => 0.18 / 0.12 for step 5
training_accuracy / validation_accuracy => 0.14 / 0.12 for step 6
training_accuracy / validation_accuracy => 0.16 / 0.12 for step 7
training_accuracy / validation_accuracy => 0.08 / 0.12 for step 8
training_accuracy / validation_accuracy => 0.18 / 0.12 for step 9
training_accuracy / validation_accuracy => 0.12 / 0.12 for step 10
training_accuracy / validation_accuracy => 0.16 / 0.12 for step 20
training_accuracy / validation_accuracy => 0.10 / 0.06 for step 30
training_accuracy / validation_accuracy => 0.02 / 0.06 for step 40
training_accuracy / validation_accuracy => 0.16 / 0.10 for step 50
train

In [20]:
predict = tf.argmax(y_out, 1)

#进行实际测试结果的预测
predicted_labels = np.zeros(test_set.shape[0])                   #定义预测结果存放array
predicted_set_size = test_set.shape[0]
for i in range(0, (int)(predicted_set_size / BATCH_SIZE)):
    predicted_labels[i * BATCH_SIZE : (i+1) *BATCH_SIZE] = predict.eval(feed_dict = {x: test_set[i * BATCH_SIZE : (i+1) *BATCH_SIZE], keep_prob: 1.0})
    
print('predicted_lables({0})'.format(len(predicted_labels)))

# save results
np.savetxt('submission_softmax.csv', 
           np.c_[range(1,len(test_set)+1),predicted_labels], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt='%d')

predicted_lables(28000)
